In [1]:
import os
import shutil

# 1) Define repo URL and local directory name
REPO_URL = "https://token@github.com/semilleroCV/breastcatt-commit-97752a8.git"
REPO_DIR = "breastcatt-commit-97752a8"

# 2) Delete the directory if it exists
if os.path.isdir(REPO_DIR):
    print(f"Directory '{REPO_DIR}' exists. Deleting it...")
    shutil.rmtree(REPO_DIR)

# 3) Clone the repository
print(f"Cloning repository into ./{REPO_DIR}...")
os.system(f"git clone {REPO_URL}")

# 4) Change into the repo directory
os.chdir(REPO_DIR)
print("Current working directory:", os.getcwd())

Cloning repository into ./breastcatt-commit-97752a8...
Current working directory: /content/breastcatt-commit-97752a8


In [2]:
# every time you do a factory reset you have to run this
!pip install --upgrade -r requirements.txt -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.9/981.9 kB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 106.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 127.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 128.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [ ]:
# IF YOU ARE WORKING ON KAGGLE OR GOOGLE COLAB DO NOT EXECUTE THIS CELL

# import sys
# import os

# # The notebook is in ‘notebooks/’, but the project (and `breastcatt`) is in the parent directory.
# # We add the parent directory to the path so we can import `breastcatt`.
# project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
# if project_root not in sys.path:
#   sys.path.append(project_root)

In [3]:
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, Resize, ToTensor, Lambda
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score
from torchmetrics.classification import BinarySpecificity, MulticlassSpecificity
from huggingface_hub import hf_hub_download
# from safetensors.torch import load_file

import numpy as np
import json

from breastcatt import tfvit

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def evaluate_split(model, dataloader, device):
    """
    Evaluates the model on a given dataloader and computes metrics.
    Handles both binary (num_classes=1) and multi-class (num_classes>1) cases.
    """
    model.eval()
    all_preds = []
    all_labels = []

    # Initialize metrics based on model's output shape
    is_binary = (model.num_classes == 1)

    if is_binary:
        specificity_metric = BinarySpecificity().to(device)
    else:
        # For multi-class, we'll compute metrics per class then macro-average
        specificity_metric = MulticlassSpecificity(num_classes=model.num_classes, average='macro').to(device)

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
            labels = batch["labels"]
            outputs = model(**batch)

            if is_binary:
                # Get probabilities and predictions
                probs = outputs.logits.sigmoid()  # shape [B, 1]
                preds = (probs > 0.5).long().squeeze(-1)  # shape [B]

                # Update metrics with proper shapes
                specificity_metric.update(probs.squeeze(-1), labels)  # [B], [B]
            else:
                # Multi-class case
                preds = outputs.logits.argmax(dim=-1)  # shape [B]
                specificity_metric.update(outputs.logits, labels)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Compute metrics
    if is_binary:
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, zero_division=0)
        sensitivity = recall_score(all_labels, all_preds, zero_division=0)
        specificity = specificity_metric.compute().item()
    else:
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
        sensitivity = recall_score(all_labels, all_preds, average='macro', zero_division=0)
        specificity = specificity_metric.compute().item()

    return {
        "accuracy": accuracy,
        "precision": precision,
        "sensitivity": sensitivity,
        "specificity": specificity,
    }

# from datasets import load_dataset

# dataset = load_dataset("SemilleroCV/DMR-IR", revision="69ffd6240b4a50bc4a05c59b70773f3a506054f2")

# train_val_split = 0.15

# # If we don't have a validation split, split off a percentage of train as validation.
# train_val_split = None if "validation" in dataset.keys() else train_val_split
# if isinstance(train_val_split, float) and train_val_split > 0.0:
#     split = dataset["train"].train_test_split(train_val_split, seed=0)
#     dataset["train"] = split["train"]
#     dataset["validation"] = split["test"]

# print(f"validation split size: {len(dataset['validation'])}")

In [4]:
import os
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from collections import defaultdict
from datasets import Image as HFImage

# Load dataset without decoding images to access filenames
dataset = load_dataset("SemilleroCV/DMR-IR", revision="69ffd6240b4a50bc4a05c59b70773f3a506054f2",
                        token='token')
dataset = dataset.cast_column("image", HFImage(decode=False))

# Extract patient ID from image path
def get_patient_id(example):
    filename = os.path.basename(example['image']['path'])
    patient_id = filename.split("_")[1]  # e.g., PAC_17_023 → '17'
    return {'patient_id': patient_id}

# Apply to training set only
dataset['train'] = dataset['train'].map(get_patient_id)

# decode images to be able work with them later
dataset = dataset.cast_column("image", HFImage(decode=True))

# Group samples by patient
patient_to_indices = defaultdict(list)
for idx, ex in enumerate(dataset['train']):
    patient_to_indices[ex['patient_id']].append(idx)

# Get list of patient IDs
all_patient_ids = list(patient_to_indices.keys())

# Split patient IDs (stratify if needed, e.g., by label)
train_patients, val_patients = train_test_split(
    all_patient_ids, test_size=0.15, random_state=0
)

# Reconstruct splits based on patient ID
train_indices, val_indices = [], []
for pid in train_patients:
    train_indices.extend(patient_to_indices[pid])
for pid in val_patients:
    val_indices.extend(patient_to_indices[pid])

# Create new splits
dataset_train = dataset['train'].select(train_indices)
dataset_val = dataset['train'].select(val_indices)

# Replace original train and add validation
dataset['train'] = dataset_train
dataset['validation'] = dataset_val

# Optional: check for patient leakage
train_patients_set = set(dataset['train']['patient_id'])
val_patients_set = set(dataset['validation']['patient_id'])
leakage = train_patients_set & val_patients_set
print(f"Patient leakage: {len(leakage)} patients")  # should be 0

# Check final counts
print(f"Train images: {len(dataset['train'])}, Train patients: {len(train_patients)}")
print(f"Validation images: {len(dataset['validation'])}, Validation patients: {len(val_patients)}")

README.md:   0%|          | 0.00/6.22k [00:00<?, ?B/s]

train-00000-of-00014.parquet:   0%|          | 0.00/278M [00:00<?, ?B/s]

train-00001-of-00014.parquet:   0%|          | 0.00/274M [00:00<?, ?B/s]

train-00002-of-00014.parquet:   0%|          | 0.00/260M [00:00<?, ?B/s]

train-00003-of-00014.parquet:   0%|          | 0.00/275M [00:00<?, ?B/s]

...

test-00003-of-00004.parquet:   0%|          | 0.00/298M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5663 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1470 [00:00<?, ? examples/s]

Map:   0%|          | 0/5663 [00:00<?, ? examples/s]

Patient leakage: 0 patients
Train images: 4831, Train patients: 187
Validation images: 832, Validation patients: 33


In [5]:
# --- ONLY RUN THIS CELL to Load one of our models ---
MODEL_REPO_ID = "SemilleroCV/tfvit-base-text-dmr-ir" # it can be base, base-text and base-text-seg

print(f"Loading model from {MODEL_REPO_ID}...")
config_path = hf_hub_download(repo_id=MODEL_REPO_ID, filename="config.json", revision="d3f752bd5b80e2389546f1bf3473ff0768bac02a")
try:
    state_dict_path = hf_hub_download(repo_id=MODEL_REPO_ID, filename="pytorch_model.bin",
                                    revision="d3f752bd5b80e2389546f1bf3473ff0768bac02a"
    )
except:
    state_dict_path = hf_hub_download(repo_id=MODEL_REPO_ID, filename="model.safetensors",
                                    revision="d3f752bd5b80e2389546f1bf3473ff0768bac02a"
    )

# Load config
with open(config_path, "r") as f:
    init_args = json.load(f)

# Init model
model = (tfvit.multimodal_vit_large_patch16(**init_args) if init_args["embed_dim"] == 1024
         else tfvit.multimodal_vit_base_patch16(**init_args))

# Load and adapt state dict
state_dict = load_file(state_dict_path, device="cpu") if state_dict_path.endswith("safetensors") \
            else torch.load(state_dict_path, map_location="cpu")

# Key renaming
state_dict = {k.replace("text_embed_proj.", "language_model.proj."): v
              for k, v in state_dict.items()}

# Load weights (ignore missing LM weights)
model.load_state_dict(state_dict, strict=False)
model.to(DEVICE)

# --- Transformations ---
min_max_norm = Lambda(lambda x: (x - x.min()) / (x.max() - x.min() + 1e-8) if x.max() > x.min() else x)
val_transforms = Compose(
    [
        Resize((224, 224)),
        ToTensor(),
        min_max_norm,
    ]
)

def preprocess_val(example_batch):
    """Apply _val_transforms across a batch."""
    example_batch["pixel_values"] = [
        val_transforms(image) for image in example_batch["image"]
    ]
    return example_batch

# DataLoaders creation:
def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    texts = [example["text"] for example in examples]
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "texts": texts, "labels": labels}

eval_dataset = dataset["validation"].with_transform(preprocess_val)

BATCH_SIZE = 512
test_dataloader = DataLoader(dataset = eval_dataset, batch_size = BATCH_SIZE,
                             shuffle = False, collate_fn = collate_fn)

metrics = evaluate_split(model, test_dataloader, DEVICE)

print(f"  Accuracy:    {metrics['accuracy']:.4f}")
print(f"  Precision:   {metrics['precision']:.4f}")
print(f"  Sensitivity: {metrics['sensitivity']:.4f}")
print(f"  Specificity: {metrics['specificity']:.4f}")

Loading model from SemilleroCV/tfvit-base-text-dmr-ir...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/175 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/461M [00:00<?, ?B/s]

📥 Downloading MAE base weights from Facebook's servers...
URL: https://dl.fbaipublicfiles.com/mae/pretrain/mae_pretrain_vit_base.pth
Saving to: checkpoints/fvit/mae_pretrain_vit_base.pth
Progress: 100.0%
✅ Successfully downloaded MAE base weights to: checkpoints/fvit/mae_pretrain_vit_base.pth


...

pytorch_model.bin:   0%|          | 0.00/713M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/713M [00:00<?, ?B/s]

Loading checkpoint from: checkpoints/fvit/mae_pretrain_vit_base.pth
Adapting patch_embed.proj.weight from 3 channels to 1 channel...

✅ Loaded weights: 148 layers.
❌ Not found in model: 0 layers.
📋 Details of load_state_dict:
_IncompatibleKeys(missing_keys=['cls_token', 'pos_embed', 'language_model.model_lm.embeddings.word_embeddings.weight', 'language_model.model_lm.embeddings.position_embeddings.weight', 'language_model.model_lm.embeddings.token_type_embeddings.weight', 'language_model.model_lm.encoder.layer.0.attention.ln.weight', 'language_model.model_lm.encoder.layer.0.attention.ln.bias', 'language_model.model_lm.encoder.layer.0.attention.self.query.weight', 'language_model.model_lm.encoder.layer.0.attention.self.query.bias', 'language_model.model_lm.encoder.layer.0.attention.self.key.weight', 'language_model.model_lm.encoder.layer.0.attention.self.key.bias', 'language_model.model_lm.encoder.layer.0.attention.self.value.weight', 'language_model.model_lm.encoder.layer.0.attention.s

Evaluating: 100%|██████████| 2/2 [00:29<00:00, 14.50s/it]

  Accuracy:    0.4087
  Precision:   0.2043
  Sensitivity: 0.5000
  Specificity: 0.5000


In [6]:
# --- ONLY RUN THIS CELL to Load one of our models ---
MODEL_REPO_ID = "SemilleroCV/tfvit-base-text-seg-2" # it can be base, base-text and base-text-seg

print(f"Loading model from {MODEL_REPO_ID}...")
config_path = hf_hub_download(repo_id=MODEL_REPO_ID, filename="config.json")
try:
    state_dict_path = hf_hub_download(repo_id=MODEL_REPO_ID, filename="pytorch_model.bin")
except:
    state_dict_path = hf_hub_download(repo_id=MODEL_REPO_ID, filename="model.safetensors")

# Load config
with open(config_path, "r") as f:
    init_args = json.load(f)

# Init model
model = (tfvit.multimodal_vit_large_patch16(**init_args) if init_args["embed_dim"] == 1024
         else tfvit.multimodal_vit_base_patch16(**init_args))

# Load and adapt state dict
state_dict = load_file(state_dict_path, device="cpu") if state_dict_path.endswith("safetensors") \
            else torch.load(state_dict_path, map_location="cpu")

# Key renaming
state_dict = {k.replace("text_embed_proj.", "language_model.proj."): v
              for k, v in state_dict.items()}

# Load weights (ignore missing LM weights)
model.load_state_dict(state_dict)
model.to(DEVICE)

# --- Transformations ---
min_max_norm = Lambda(lambda x: (x - x.min()) / (x.max() - x.min() + 1e-8) if x.max() > x.min() else x)
val_transforms = Compose(
    [
        Resize((224, 224)),
        ToTensor(),
        min_max_norm,
    ]
)

def preprocess_val(example_batch):
    """Apply _val_transforms across a batch."""
    example_batch["pixel_values"] = [
        val_transforms(image) for image in example_batch["image"]
    ]
    return example_batch

# DataLoaders creation:
def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    texts = [example["text"] for example in examples]
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "texts": texts, "labels": labels}

eval_dataset = dataset["validation"].with_transform(preprocess_val)

BATCH_SIZE = 64
test_dataloader = DataLoader(dataset = eval_dataset, batch_size = BATCH_SIZE,
                             shuffle = False, collate_fn = collate_fn)

metrics = evaluate_split(model, test_dataloader, DEVICE)

print(f"  Accuracy:    {metrics['accuracy']:.4f}")
print(f"  Precision:   {metrics['precision']:.4f}")
print(f"  Sensitivity: {metrics['sensitivity']:.4f}")
print(f"  Specificity: {metrics['specificity']:.4f}")

Loading model from SemilleroCV/tfvit-base-text-seg-2...


config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.30G [00:00<?, ?B/s]

✅ MAE base weights already exist at: checkpoints/fvit/mae_pretrain_vit_base.pth
Error loading weights from checkpoints/segmentation/lucky-sweep-6_0.4937.pth: [Errno 2] No such file or directory: 'checkpoints/segmentation/lucky-sweep-6_0.4937.pth'


FileNotFoundError: [Errno 2] No such file or directory: 'checkpoints/segmentation/lucky-sweep-6_0.4937.pth'

In [7]:
# --- ONLY RUN THIS CELL to Load one of our models ---
MODEL_REPO_ID = "SemilleroCV/tfvit-base-text-2" # it can be base, base-text and base-text-seg

print(f"Loading model from {MODEL_REPO_ID}...")
config_path = hf_hub_download(repo_id=MODEL_REPO_ID, filename="config.json")
try:
    state_dict_path = hf_hub_download(repo_id=MODEL_REPO_ID, filename="pytorch_model.bin")
except:
    state_dict_path = hf_hub_download(repo_id=MODEL_REPO_ID, filename="model.safetensors")

# Load config
with open(config_path, "r") as f:
    init_args = json.load(f)

# Init model
model = (tfvit.multimodal_vit_large_patch16(**init_args) if init_args["embed_dim"] == 1024
         else tfvit.multimodal_vit_base_patch16(**init_args))

# Load and adapt state dict
state_dict = load_file(state_dict_path, device="cpu") if state_dict_path.endswith("safetensors") \
            else torch.load(state_dict_path, map_location="cpu")

# Key renaming
state_dict = {k.replace("text_embed_proj.", "language_model.proj."): v
              for k, v in state_dict.items()}

# Load weights (ignore missing LM weights)
model.load_state_dict(state_dict, strict=False)
model.to(DEVICE)

# --- Transformations ---
min_max_norm = Lambda(lambda x: (x - x.min()) / (x.max() - x.min() + 1e-8) if x.max() > x.min() else x)
val_transforms = Compose(
    [
        Resize((224, 224)),
        ToTensor(),
        min_max_norm,
    ]
)

def preprocess_val(example_batch):
    """Apply _val_transforms across a batch."""
    example_batch["pixel_values"] = [
        val_transforms(image) for image in example_batch["image"]
    ]
    return example_batch

# DataLoaders creation:
def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    texts = [example["text"] for example in examples]
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "texts": texts, "labels": labels}

eval_dataset = dataset["validation"].with_transform(preprocess_val)

BATCH_SIZE = 512
test_dataloader = DataLoader(dataset = eval_dataset, batch_size = BATCH_SIZE,
                             shuffle = False, collate_fn = collate_fn)

metrics = evaluate_split(model, test_dataloader, DEVICE)

print(f"  Accuracy:    {metrics['accuracy']:.4f}")
print(f"  Precision:   {metrics['precision']:.4f}")
print(f"  Sensitivity: {metrics['sensitivity']:.4f}")
print(f"  Specificity: {metrics['specificity']:.4f}")

Loading model from SemilleroCV/tfvit-base-text-2...


config.json:   0%|          | 0.00/175 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

✅ MAE base weights already exist at: checkpoints/fvit/mae_pretrain_vit_base.pth
Loading checkpoint from: checkpoints/fvit/mae_pretrain_vit_base.pth
Adapting patch_embed.proj.weight from 3 channels to 1 channel...

✅ Loaded weights: 148 layers.
❌ Not found in model: 0 layers.
📋 Details of load_state_dict:
_IncompatibleKeys(missing_keys=['cls_token', 'pos_embed', 'language_model.model_lm.embeddings.word_embeddings.weight', 'language_model.model_lm.embeddings.position_embeddings.weight', 'language_model.model_lm.embeddings.token_type_embeddings.weight', 'language_model.model_lm.encoder.layer.0.attention.ln.weight', 'language_model.model_lm.encoder.layer.0.attention.ln.bias', 'language_model.model_lm.encoder.layer.0.attention.self.query.weight', 'language_model.model_lm.encoder.layer.0.attention.self.query.bias', 'language_model.model_lm.encoder.layer.0.attention.self.key.weight', 'language_model.model_lm.encoder.layer.0.attention.self.key.bias', 'language_model.model_lm.encoder.layer.0.at

Evaluating: 100%|██████████| 2/2 [00:29<00:00, 14.60s/it]

  Accuracy:    0.9784
  Precision:   0.9792
  Sensitivity: 0.9676
  Specificity: 0.9858


## evaluating on test

In [8]:
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, Resize, ToTensor, Lambda
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score
from torchmetrics.classification import BinarySpecificity, MulticlassSpecificity
from huggingface_hub import hf_hub_download
# from safetensors.torch import load_file

import numpy as np
import json

from breastcatt import tfvit

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def evaluate_split(model, dataloader, device):
    """
    Evaluates the model on a given dataloader and computes metrics.
    Handles both binary (num_classes=1) and multi-class (num_classes>1) cases.
    """
    model.eval()
    all_preds = []
    all_labels = []

    # Initialize metrics based on model's output shape
    is_binary = (model.num_classes == 1)

    if is_binary:
        specificity_metric = BinarySpecificity().to(device)
    else:
        # For multi-class, we'll compute metrics per class then macro-average
        specificity_metric = MulticlassSpecificity(num_classes=model.num_classes, average='macro').to(device)

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
            labels = batch["labels"]
            outputs = model(**batch)

            if is_binary:
                # Get probabilities and predictions
                probs = outputs.logits.sigmoid()  # shape [B, 1]
                preds = (probs > 0.5).long().squeeze(-1)  # shape [B]

                # Update metrics with proper shapes
                specificity_metric.update(probs.squeeze(-1), labels)  # [B], [B]
            else:
                # Multi-class case
                preds = outputs.logits.argmax(dim=-1)  # shape [B]
                specificity_metric.update(outputs.logits, labels)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Compute metrics
    if is_binary:
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, zero_division=0)
        sensitivity = recall_score(all_labels, all_preds, zero_division=0)
        specificity = specificity_metric.compute().item()
    else:
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
        sensitivity = recall_score(all_labels, all_preds, average='macro', zero_division=0)
        specificity = specificity_metric.compute().item()

    return {
        "accuracy": accuracy,
        "precision": precision,
        "sensitivity": sensitivity,
        "specificity": specificity,
    }

from datasets import load_dataset

dataset = load_dataset("SemilleroCV/DMR-IR", revision="69ffd6240b4a50bc4a05c59b70773f3a506054f2", split='test')

Using the latest cached version of the dataset since SemilleroCV/DMR-IR couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/SemilleroCV___dmr-ir/default/0.0.0/69ffd6240b4a50bc4a05c59b70773f3a506054f2 (last modified on Sun Jul 27 21:28:12 2025).


In [ ]:
# --- ONLY RUN THIS CELL to Load one of our models ---
MODEL_REPO_ID = "SemilleroCV/tfvit-base-text-2" # it can be base, base-text and base-text-seg

print(f"Loading model from {MODEL_REPO_ID}...")
config_path = hf_hub_download(repo_id=MODEL_REPO_ID, filename="config.json")
try:
    state_dict_path = hf_hub_download(repo_id=MODEL_REPO_ID, filename="pytorch_model.bin")
except:
    state_dict_path = hf_hub_download(repo_id=MODEL_REPO_ID, filename="model.safetensors")

# Load config
with open(config_path, "r") as f:
    init_args = json.load(f)

# Init model
model = (tfvit.multimodal_vit_large_patch16(**init_args) if init_args["embed_dim"] == 1024
         else tfvit.multimodal_vit_base_patch16(**init_args))

# Load and adapt state dict
state_dict = load_file(state_dict_path, device="cpu") if state_dict_path.endswith("safetensors") \
            else torch.load(state_dict_path, map_location="cpu")

# Key renaming
state_dict = {k.replace("text_embed_proj.", "language_model.proj."): v
              for k, v in state_dict.items()}

# Load weights (ignore missing LM weights)
model.load_state_dict(state_dict, strict=False)
model.to(DEVICE)

# --- Transformations ---
min_max_norm = Lambda(lambda x: (x - x.min()) / (x.max() - x.min() + 1e-8) if x.max() > x.min() else x)
val_transforms = Compose(
    [
        Resize((224, 224)),
        ToTensor(),
        min_max_norm,
    ]
)

def preprocess_val(example_batch):
    """Apply _val_transforms across a batch."""
    example_batch["pixel_values"] = [
        val_transforms(image) for image in example_batch["image"]
    ]
    return example_batch

# DataLoaders creation:
def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    texts = [example["text"] for example in examples]
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "texts": texts, "labels": labels}

eval_dataset = dataset.with_transform(preprocess_val)

BATCH_SIZE = 512
test_dataloader = DataLoader(dataset = eval_dataset, batch_size = BATCH_SIZE,
                             shuffle = False, collate_fn = collate_fn)

metrics = evaluate_split(model, test_dataloader, DEVICE)

print(f"  Accuracy:    {metrics['accuracy']:.4f}")
print(f"  Precision:   {metrics['precision']:.4f}")
print(f"  Sensitivity: {metrics['sensitivity']:.4f}")
print(f"  Specificity: {metrics['specificity']:.4f}")

Loading model from SemilleroCV/tfvit-base-text-2...
✅ MAE base weights already exist at: checkpoints/fvit/mae_pretrain_vit_base.pth
Loading checkpoint from: checkpoints/fvit/mae_pretrain_vit_base.pth
Adapting patch_embed.proj.weight from 3 channels to 1 channel...

✅ Loaded weights: 148 layers.
❌ Not found in model: 0 layers.
📋 Details of load_state_dict:
_IncompatibleKeys(missing_keys=['cls_token', 'pos_embed', 'language_model.model_lm.embeddings.word_embeddings.weight', 'language_model.model_lm.embeddings.position_embeddings.weight', 'language_model.model_lm.embeddings.token_type_embeddings.weight', 'language_model.model_lm.encoder.layer.0.attention.ln.weight', 'language_model.model_lm.encoder.layer.0.attention.ln.bias', 'language_model.model_lm.encoder.layer.0.attention.self.query.weight', 'language_model.model_lm.encoder.layer.0.attention.self.query.bias', 'language_model.model_lm.encoder.layer.0.attention.self.key.weight', 'language_model.model_lm.encoder.layer.0.attention.self.ke

Evaluating: 100%|██████████| 3/3 [00:54<00:00, 18.20s/it]

  Accuracy:    0.8762
  Precision:   0.8333
  Sensitivity: 0.8364
  Specificity: 0.9000
